In [14]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import grad
import matplotlib

In [15]:
#n_qubits = 17
dev = qml.device("default.qubit", wires=n_qubits)

# Quantum Layer: First LQ Embed Layer
def lq_embed_1(theta, theta_par):
    for h in range(n_qubits):
        qml.Hadamard(wires=h)
    for i in range(n_qubits):
        qml.RX(theta[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i+1])
    for i in range(n_qubits):
        qml.RY(theta_par[i], wires=i)

# Quantum Layer: Subsequent Layers
def lq_embed_m(theta, theta_par):
    for i in range(n_qubits):
        qml.RX(theta[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i+1])
    for i in range(n_qubits):
        qml.RY(theta_par[i], wires=i)

# Full Layerwise Ansatz
@qml.qnode(dev, interface="autograd")
def critic_qnode(inputs, weights):
    lq_embed_1(inputs, weights[0])
    for m in range(1, len(weights)):
        lq_embed_m(inputs, weights[m])
    return qml.expval(qml.PauliZ(0))

In [16]:
import jax
import pennylane as qml

print("JAX version:", jax.__version__)
print("PennyLane version:", qml.__version__)

JAX version: 0.4.25
PennyLane version: 0.37.0


In [17]:
@qml.qnode(dev)
def actor_qnode(state_input, theta, theta_par):
    # Apply Hadamard
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    
    # RX with state-dependent angle
    for i in range(n_qubits):
        qml.RX(state_input[i], wires=i)
    
    # Entanglement (CZ between neighbours)
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i+1])
    
    # RY with trainable parameter
    for i in range(n_qubits):
        qml.RY(theta_par[i], wires=i)
    
    # Measurement
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [18]:
@qml.qnode(dev)
def critic_qnode(critic_input, theta, theta_par):
    # Build same ansatz as above, with critic_input
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
        qml.RX(critic_input[i], wires=i)

    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i+1])

    for i in range(n_qubits):
        qml.RY(theta_par[i], wires=i)

    # Q-value output (e.g., Z₀ expectation)
    return qml.expval(qml.PauliZ(0))

In [20]:
m_layers = 3
E_rem = 500e03
num_gus = 4
u = []
uav_pos = [0, 0, 0]
uav_v_max = 30
zeta_v = 1
uav_v = zeta_v * uav_v_max
noma_g = [1, 2, 3]
pwr_alloc = 1
action_vec = []

action_vec.append(uav_v)
for k in range(len(noma_g)):
    action_vec.append(noma_g[k])
action_vec.append(pwr_alloc)

crit_inp_vec = []

for i in range(len(uav_pos)):
    crit_inp_vec.append(uav_pos[i])
crit_inp_vec.append(E_rem)
for j in range(2*num_gus):
    u.append(j)
    crit_inp_vec.append(u[j])
crit_inp_vec.append(uav_v)
for k in range(len(noma_g)):
    crit_inp_vec.append(noma_g[k])
crit_inp_vec.append(pwr_alloc)

#full_input = np.concatenate([crit_inp_vec, action_vec])
full_input = crit_inp_vec

n_qubits = len(full_input)
print(n_qubits)

theta = np.zeros(n_qubits, requires_grad=False)
theta_par = np.random.randn(n_qubits, requires_grad=True)

# Forward pass
q_vals = actor_qnode(crit_inp_vec, theta, theta_par)
q_val = critic_qnode(full_input, theta, theta_par)

22


IndexError: list index out of range

In [12]:
critic_grad_fn = qml.grad(critic_qnode, argnum=2)
grad_vals = critic_grad_fn(full_input, theta, theta_par)

In [13]:
opt = qml.GradientDescentOptimizer(stepsize=0.1)
for _ in range(10):
    theta_par, loss = opt.step_and_cost(lambda tp: -critic_qnode(full_input, theta, tp), theta_par)
    print("Loss:", loss)

Loss: -1.1102230246251565e-16
Loss: 1.1102230246251565e-16
Loss: -1.1102230246251565e-16
Loss: 1.1102230246251565e-16
Loss: 1.1102230246251565e-16
Loss: -0.0
Loss: -1.1102230246251565e-16
Loss: -1.1102230246251565e-16
Loss: -1.1102230246251565e-16
Loss: -1.1102230246251565e-16
